In [13]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\saura\AppData\Roaming\nltk_data...


True

In [48]:
import numpy as np
import pandas as pd

In [49]:
training_data = pd.read_csv('train.csv')
testing_data = pd.read_csv('test.csv')

In [50]:
training_data['text_arr']=training_data['text'].str.split()
train_test = training_data[['text_arr','airline_sentiment']]
documents = train_test.values
documents_test = testing_data['text'].str.split().values

In [51]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [52]:
from nltk.corpus import wordnet
def get_simple_pos(tag): #creating simple tags to pass into the lemmatizer
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [53]:
from nltk.corpus import stopwords
import string
stops = stopwords.words('english') + list(string.punctuation)

In [54]:
from nltk import pos_tag

In [55]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])                                 
            clean_word = lemmatizer.lemmatize(w, get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [56]:
documents = [[clean_review(document), category] for document, category in documents]

In [57]:
documents_test=[(clean_review(document)) for document in documents_test]

In [58]:
print(type(documents))
train_data = pd.Series(documents,dtype='object')

<class 'list'>


In [59]:
X_train = train_data[0:]

In [60]:
all_words=[]
for i in documents:
    all_words+=i[0]
    
frequency=nltk.FreqDist(all_words)

common=frequency.most_common(3000)
features=[i[0]for i in common]

In [61]:
def get_feature_dict(words):
    current_features={}
    words_set=set(words)
    for w in features:
        current_features[w]=w in words_set
    return current_features

In [62]:
training_Data=[(get_feature_dict(doc),category) for doc,category in documents]

In [63]:
testing_Data=[(get_feature_dict(doc))for doc in documents_test]

In [67]:
from nltk import NaiveBayesClassifier

In [68]:
classifier=NaiveBayesClassifier.train(training_Data)

In [69]:
nltk.classify.accuracy(classifier,training_Data)

0.8375227686703096

In [71]:
predict=[]
for i in range(len(testing_data)):
    predict.append(classifier.classify(testing_Data[i]))

In [72]:
np.savetxt('predictions.csv', X=predict, delimiter=',', fmt="%s")

In [73]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier

In [74]:
svc = SVC()
classifier_sklearn = SklearnClassifier(svc)

In [75]:
classifier_sklearn.train(training_Data)

<SklearnClassifier(SVC())>

In [76]:
nltk.classify.accuracy(classifier_sklearn, training_Data)

0.906648451730419

In [77]:
from sklearn.ensemble import RandomForestClassifier

In [78]:
rfc = RandomForestClassifier()
classifier_sklearn1 = SklearnClassifier(rfc)

In [79]:
classifier_sklearn1.train(training_Data)

<SklearnClassifier(RandomForestClassifier())>

In [80]:
nltk.classify.accuracy(classifier_sklearn1, training_Data)

0.992167577413479

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(max_features = 3000)
text_col = training_data['text'].values
x_train_features = count_vec.fit_transform(text_col)
x_train_dense = x_train_features.todense()

In [ ]:
x_test_features = count_vec.transform(x_test)

In [ ]:
count_svc = SVC()

In [ ]:
y_col = training_data['airline_sentiment'].values

In [ ]:
count_svc.fit(x_train_dense,y_col)

In [ ]:
svc.score(x_train_dense , y_test)